<a href="https://colab.research.google.com/github/satyasri77/chat_bot/blob/main/llama_rag_customer_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q llama-cpp-python sentence-transformers faiss-cpu huggingface-hub

In [ ]:
pip install -U langchain-community

In [ ]:
from huggingface_hub import hf_hub_download

HF_TOKEN = "your_token"  # ← Replace with your Hugging Face token

model_path = hf_hub_download(
    repo_id="TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
    filename="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    token=HF_TOKEN
)


In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = TextLoader("customer_service_info.txt")  # ← Upload this file first
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db = FAISS.from_documents(docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 1})


In [ ]:
from llama_cpp import Llama

llm = Llama(model_path=model_path, n_ctx=1024, verbose=False)


In [ ]:
def ask(question):
    # Step 1: Retrieve top relevant chunks
    context_docs = retriever.get_relevant_documents(question)
    context = "\n".join(doc.page_content for doc in context_docs)

    # Step 2: If nothing relevant was found, return fallback message
    if not context.strip():
        return "I'm only able to answer customer support related questions."

    # Step 3: Build a strict prompt using only the retrieved context
    prompt = f"""
You are a helpful customer support assistant.
Only answer based on the context below. Do not use outside knowledge.
If the answer is not related to customer support queries, reply with:
"I'm only able to answer customer support related questions."

Context:
{context}

Question: {question}
Answer:"""

    response = llm(prompt, max_tokens=200,temperature=0,top_p=0.8, top_k=40)
    return response["choices"][0]["text"].strip()


In [ ]:
print("💬 LLaMA RAG Chatbot Ready! Type 'exit' to stop.\n")
while True:
    q = input("You: ")
    if q.lower() in ['exit', 'quit']:
        break
    print("Bot:", ask(q), "\n")